In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
from tqdm import tqdm
import pandas as pd
from PIL import Image, ImageDraw

import json

In [4]:
IMAGES_DIR = 'fddb/val/images/'
ANNOTATIONS_PATH = 'fddb/val/annotations/'
RESULT_DIR = 'fddb/val/original_ann_images/'

## Save the image name and the corresponding annotation

In [17]:
original_annotations = pd.DataFrame()

for path, subdir, files in tqdm(os.walk(ANNOTATIONS_PATH)):
    for name in files:
        boxes_of_boxes = []
        boxes = []

        with open(path + name, 'r') as f:
            ann_dict = json.load(f)
            dict_boxes = ann_dict['object']
            for box in dict_boxes:
                box = box['bndbox']
                b = [int(box['xmin']), int(box['ymin']), int(box['xmax']), int(box['ymax'])]
                boxes.append(b)

            boxes_of_boxes.append(boxes)
            df = {'image_path': IMAGES_DIR + ann_dict['filename'], 'boxes': boxes_of_boxes} 
            df = pd.DataFrame(df)
            original_annotations =  pd.concat([original_annotations, df], ignore_index = True, axis = 0)


1it [00:27, 27.57s/it]


In [18]:
original_annotations.head()

,image_path,boxes
0,fddb/val/images/2002-07-19_img_130.jpg,"[[118, 38, 179, 133]]"
1,fddb/val/images/2002-07-19_img_135.jpg,"[[69, 70, 163, 225], [231, 17, 313, 151]]"
2,fddb/val/images/2002-07-19_img_141.jpg,"[[152, 35, 217, 145]]"
3,fddb/val/images/2002-07-19_img_158.jpg,"[[124, 22, 237, 182]]"
4,fddb/val/images/2002-07-19_img_160.jpg,"[[235, 64, 300, 161]]"


Now draw these annotations on the images and save them

In [21]:
def draw_boxes_on_image(path, boxes):

    image_copy = Image.open(path)
    draw = ImageDraw.Draw(image_copy)

    for b in boxes:
        xmin, ymin, xmax, ymax= b
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
             outline=outline
        )
    return image_copy

In [23]:
for index, row in original_annotations.iterrows():
    img_path = row['image_path']
    annotated_image = draw_boxes_on_image(img_path, row['boxes'])
    
    output_path = img_path.split("/")[-1]
    output_path = RESULT_DIR + output_path
    annotated_image.save(output_path)
